# Climate variables were gathered through the NASA POWER API, offering reliable, real-time historical data for modeling.

In [4]:
# import necessary libraries
import requests
import pandas as pd
import os

In [ ]:
def fetch_nasa_power_data(latitude, longitude, start_year, end_year, parameters, filename="climate_data.csv"):

    # Define the base URL for the POWER API
    base_url = "https://power.larc.nasa.gov/api/temporal/monthly/point"

    # Add precipitation and solar radiation to the parameters
    parameters = parameters + ",PRECTOTCORR,ALLSKY_SFC_SW_DWN"

    # Required query parameters
    query_params = {
        "latitude": latitude,
        "longitude": longitude,
        "start": start_year,
        "end": end_year,
        "parameters": parameters,
        "community": "RE",
        "format": "JSON"
    }

    # API request
    response = requests.get(base_url, params=query_params)
    response.raise_for_status()

    # Parse the JSON response
    data = response.json()

    # Extract data from the response
    records = []
    parameters_list = parameters.split(',')
    for year_month, values in data['properties']['parameter'][parameters_list[0]].items():
        record = {'year_month': year_month}
        for param in parameters_list:
            record[param] = data['properties']['parameter'][param].get(year_month)
        records.append(record)

    # Convert records into DataFrame
    df = pd.DataFrame(records)

    # Convert the 'year_month' column to datetime
    df['year_month'] = pd.to_datetime(df['year_month'], format='%Y%m', errors='coerce')

    # Extracting Year and Month separately
    df['year'] = df['year_month'].dt.year
    df['month'] = df['year_month'].dt.month

    # Drop 'year_month' 
    df.drop(columns=['year_month'], inplace=True)

    # Remove rows with NaN values and reset index
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Save the cleaned DataFrame to a CSV file
    df.to_csv(filename, index=False)

    print(f"Data saved to {filename} in {os.getcwd()}")

    return df

In [8]:
# Main function to execute the script
if __name__ == "__main__":

    # Define the time range
    start_year = 2000
    end_year = 2021

    # Define the location (latitude and longitude)
    latitude = 28.3949
    longitude = 84.1240
    
    # The locations can be defined as follows for multiple countries.
    # locations = [
    # (61.5240, 105.3188),
    # (60.1282, 18.6435),   
     # (40.4637, -3.7492),...]

    # Define the desired parameters
    parameters = "WS2M,T2M,PS"

    # Fetch the data and save to CSV
    climate_data = fetch_nasa_power_data(latitude, longitude, start_year, end_year, parameters)

    # Display the cleaned data
    print(climate_data)

Data saved to climate_data.csv in c:\Users\sahil\New Project\Datagathering
     WS2M    T2M     PS  PRECTOTCORR  ALLSKY_SFC_SW_DWN    year  month
0    1.52  -2.96  66.61         0.04               3.51  2000.0    1.0
1    1.79  -4.34  66.47         0.06               3.95  2000.0    2.0
2    1.96  -0.11  66.69         0.08               4.77  2000.0    3.0
3    2.07   6.18  66.85         0.05               5.65  2000.0    4.0
4    2.07   9.79  66.88         1.45               5.11  2000.0    5.0
..    ...    ...    ...          ...                ...     ...    ...
259  1.74  11.60  66.89        20.41               3.95  2021.0    8.0
260  1.46  10.28  67.09         9.81               4.26  2021.0    9.0
261  1.33   7.30  67.11         7.55               4.29  2021.0   10.0
262  1.23   2.66  66.94         0.01               4.12  2021.0   11.0
263  1.27  -1.77  66.89         1.29               3.09  2021.0   12.0

[264 rows x 7 columns]
